# Импорт

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from Classes.Regression import Regression
from Classes.RegressionTeacher import RegressionTeacher
from Functions.lossFunctions import euclideError, euclideErrorGradient
from Classes.Polinomizer import Polinomizer
from Functions.gradientDescentAlgorithm import gradientDescentAlgorithm
from Functions.errorFunctions import MAE, MAPE, MSE, RMSE, R2
from Functions.regulatorFinctions import Lasso, Ridge, RidgeGrad
from sklearn.metrics import r2_score


# Загрузка датасета для регрессии.

In [25]:
models = pd.read_csv('../Data/DataLaba1/energy_task_moded.csv')
model = models.drop('year', axis=1)
model = (model-model.mean())/model.std()
model


,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,day,month,time
0,-0.367666,3.301180,-1.118616,1.843774,-0.520398,1.077616,-1.235032,1.686087,-0.908194,1.510576,...,0.958043,-0.154025,-2.983203,0.824355,1.207663,2.092365,0.366845,-0.584529,-1.567533,0.738423
1,-0.367666,3.301180,-1.118616,1.616766,-0.520398,1.060983,-1.235032,1.704523,-0.908194,1.608892,...,0.965270,-0.176708,-2.969656,0.824355,1.071676,1.767230,0.343008,-0.584529,-1.567533,0.762352
2,-0.465204,3.301180,-1.118616,1.517921,-0.520398,1.037367,-1.235032,1.748563,-0.944091,1.585227,...,0.950816,-0.197500,-2.956109,0.824355,0.935690,1.442094,0.319172,-0.584529,-1.567533,0.786280
3,-0.465204,4.561263,-1.118616,1.459284,-0.520398,1.028332,-1.235032,1.769047,-0.962039,1.546747,...,0.926726,-0.220182,-2.942562,0.824355,0.799703,1.116958,0.295335,-0.584529,-1.567533,0.810209
4,-0.367666,4.561263,-1.118616,1.526298,-0.520398,1.013546,-1.235032,1.769047,-0.962039,1.502111,...,0.926726,-0.242865,-2.929015,0.824355,0.663716,0.791822,0.271499,-0.584529,-1.567533,0.834138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19730,0.022483,-0.479068,2.415901,1.583259,2.530254,0.396436,2.458683,0.590195,1.881979,1.515963,...,1.261577,2.889209,-0.043505,-1.620295,-0.288190,-1.243811,2.273765,1.297945,1.422032,0.786280
19731,-0.075055,-0.479068,2.374391,1.568181,2.468238,0.409813,2.425451,0.608630,1.881979,1.515963,...,1.261577,2.870307,-0.043505,-1.597867,-0.220197,-1.173129,2.273765,1.297945,1.422032,0.810209
19732,1.680617,0.781015,2.374391,1.592473,2.411042,0.579492,2.383911,0.752018,1.881979,1.548286,...,1.261577,2.851405,-0.043505,-1.575439,-0.152204,-1.102447,2.273765,1.297945,1.422032,0.834138
19733,3.143676,0.781015,2.374391,1.691318,2.313197,0.645392,2.304155,0.629114,1.881979,1.562139,...,1.268202,2.813601,-0.043505,-1.553011,-0.084210,-1.031766,2.249928,1.297945,1.422032,0.858067


# Разбиваем на выборку

In [26]:
X_train, X_test, y_train, y_test = train_test_split(models.drop('Appliances', axis=1), models['Appliances'], test_size=0.5)
X_train1, X_test1, y_train1, y_test1 = train_test_split(
    model.drop('Appliances', axis=1), model['Appliances'], test_size=0.5)


# Линейная регрессия

In [58]:
function_optimizer_error = lambda td, cd, w: gradientDescentAlgorithm(
    td=td, 
    cd=cd, 
    w=w, 
    lost_function=euclideError, 
    gradient=euclideErrorGradient, 
    nsteps=1008, 
    merror=0.001, 
    step=0.00000001)


regressor = Regression()
trainer = RegressionTeacher(regressor, function_optimizer_error)
trainer.teach(np.array(X_train, dtype=np.float64), np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(X_test)

my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test)),
                                    }

print(my_line_regression_classic_errors)
print(y_predicted.tolist())


{'MAE': 134.87333747803126, 'RMSE': 165.7484426709732, 'MSE': 27472.54624785289, 'MAPE': 2.0600338897531523, 'R^2': -6.344396224167059}
[-45.26292416098357, 135.80530486806302, 126.730675485138, 32.10351034616153, 132.89075117126382, 24.47813500039524, 25.99908452298621, 239.96699772762736, -66.17706474716312, -3.2608824936194196, -6.377952457627373, 7.2485791780622435, 188.15614978860543, 24.299871399576375, 242.6622682936058, 245.53728782126746, 270.9315736410152, -43.86380974887004, 170.12285037675912, -40.77698775843385, -69.45569961303198, 135.03247905162982, 87.47365688537724, 168.9837950062763, -18.801766132906323, -54.55383333956797, 110.50559441241865, 32.77265301467304, 252.03555804044467, 279.5282682039817, -30.395633028197437, 45.47370692003605, 228.6364919708858, 231.71411835832333, -0.7614771107721481, 132.82702581953325, 52.04677380106335, 221.55626935106613, 176.41159968563736, -54.028155741499575, 242.74193122670593, 118.92046602312053, 63.07407946754927, 208.830379054

In [57]:
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=euclideError,
    gradient=euclideErrorGradient,
    nsteps=5,
    merror=0.001,
    step=0.001)


regressor = Regression()
trainer = RegressionTeacher(regressor, function_optimizer_error)
trainer.teach(np.array(X_train1, dtype=np.float64),
              np.array(y_train1, dtype=np.float64))

y_predicted = regressor.predict(X_test1)

my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test1),
                                     'RMSE': RMSE(y_predicted, y_test1),
                                     'MSE': MSE(y_predicted, y_test1),
                                     'MAPE': MAPE(y_predicted, y_test1),
                                     'R^2': R2(y_predicted, np.array(y_test1))
                                     }

print(my_line_regression_classic_errors)
print(y_predicted.tolist())


{'MAE': 1.375015632198751, 'RMSE': 1.7643762374601264, 'MSE': 3.1130235073139523, 'MAPE': 6.920142057054818, 'R^2': 0.9989809104580356}
[-0.6490007450464909, -1.1791711506401448, 4.068612063524613, -0.4589591691358017, -1.5407737242035073, 0.2319991880830361, 2.6280202976498392, -0.3946073960169935, 2.499916586571799, -0.40741994758453176, 1.0286208060942705, 1.2740367133520074, -0.23871505206043586, -2.859300816247818, 2.1317974555689974, -1.19888233849737, -2.517607592446649, 0.43104494918557695, -2.7687861274936685, -0.638622607784609, 1.8020330853884072, 0.694661889392366, -0.8453733604858467, -1.8420098525453648, 0.924711138980497, 1.1139349490550847, 2.1332139374315378, 0.18162003970274004, -1.7112414517110899, -0.6990366052253141, -1.1035503857776796, 0.5535925996105573, 0.6959169720211669, 0.7325908821551954, -1.2913915407400836, 1.2214669630956179, -0.5983095905920086, -1.6493510077040163, 0.7836676304930822, 1.6678416889475092, 1.5163460075029889, 0.2319885541925101, 2.458778

In [63]:
alpha = 0.5
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=lambda td, cd, w: euclideError(td, cd, w) + Lasso(w, alpha),
    gradient=lambda td, cd, w: euclideErrorGradient(
        td, cd, w) + alpha * w.shape[0],
    nsteps=1000,
    merror=0.01,
    step=0.00000000001)


regressor = Regression()
trainer = RegressionTeacher(regressor, function_optimizer_error)
trainer.teach(np.array(X_train, dtype=np.float64),
              np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(X_test)
# y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                     }

# my_line_regression_classic_errors
print(my_line_regression_classic_errors)
print(y_predicted.tolist())


{'MAE': 1676.8660216020012, 'RMSE': 1681.2783276916173, 'MSE': 2826696.815165521, 'MAPE': 27.248224333042508, 'R^2': -77751.92423238592}
[1912.0993042502664, 1763.0990411474943, 1774.1684720348735, 1804.9450323223468, 1741.928956568638, 1851.312868403327, 1816.7587297089785, 1669.9127263158832, 1922.2189134963483, 1866.7249446962574, 1832.9569741514028, 1871.4329150426709, 1709.0470374469157, 1807.1822737550613, 1676.861635545795, 1658.0897177109441, 1640.9925753523642, 1864.9988810999748, 1721.1469505249665, 1872.0960987523044, 1929.479858236074, 1770.6437717508422, 1755.1776841863107, 1729.7609519484615, 1880.7219991812844, 1903.0297416419796, 1742.9022366670333, 1846.7002624194815, 1658.853314394306, 1625.345287799583, 1844.1990522994556, 1815.09770850754, 1669.9722755815792, 1693.6497263683746, 1841.2024790153203, 1742.1591559608946, 1789.7125083823341, 1706.2571704258269, 1763.501296020873, 1915.540264506641, 1664.718534182414, 1776.9822480515436, 1816.102569871337, 1672.878736020

In [65]:
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=lambda td, cd, w: euclideError(td, cd, w) + Ridge(w, 1),
    gradient=lambda td, cd, w: euclideErrorGradient(td, cd, w) + RidgeGrad(w, 1),
    nsteps=100,
    merror=0.01,
    step=0.000000000001)


regressor = Regression()
trainer = RegressionTeacher(regressor, function_optimizer_error)
trainer.teach(np.array(X_train, dtype=np.float64),
              np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(X_test)
# y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                     }

my_line_regression_classic_errors
print(my_line_regression_classic_errors)
print(y_predicted.tolist())


{'MAE': 1814.874872744885, 'RMSE': 1854.252138661575, 'MSE': 3438250.9937310247, 'MAPE': 28.58797651462592, 'R^2': -115034.90850404091}
[2510.9948776317547, 1792.430669921461, 1904.0234627671343, 2093.206073445072, 1841.8195436965984, 2234.602691020456, 2128.7010618412696, 1411.854797466011, 2537.7860619894955, 2290.8754210275565, 2233.2348500371468, 2279.4558827743576, 1601.585019078771, 2107.067870664925, 1485.6066389154064, 1416.094380687967, 1342.6818036383752, 2334.2624231287355, 1697.503909717402, 2335.7515442322183, 2540.135243764488, 1853.5951642212299, 1962.5108865597404, 1672.8913837413893, 2348.0008203550715, 2465.6787265496578, 1827.8110944198088, 2171.0846625373624, 1440.4278972745092, 1299.6652435424442, 2326.9405934283077, 2142.388620859822, 1500.908123972797, 1557.6365255118822, 2211.3806638179035, 1771.41831168685, 1996.721872446229, 1576.3754481561218, 1789.4757062862816, 2531.230326480062, 1437.0331872045308, 1889.6113429214872, 2102.4727449827687, 1521.22007053154, 

In [ ]:
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=lambda td, cd, w: euclideError(td, cd, w),
    gradient=lambda td, cd, w: euclideErrorGradient(td, cd, w),
    nsteps=3000,
    merror=0.0001,
    step=0.00000000000000000000001)


regressor = Regression()
trainer = RegressionTeacher(regressor, function_optimizer_error)
trainer.teach(Polinomizer(np.array(X_train, dtype=np.float64), 3).polinomize(),
              np.array(y_train, dtype=np.float64))

y_predicted = regressor.predict(Polinomizer(np.array(X_test, dtype=np.float64), 3).polinomize())
y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test),
                                     'RMSE': RMSE(y_predicted, y_test),
                                     'MSE': MSE(y_predicted, y_test),
                                     'MAPE': MAPE(y_predicted, y_test),
                                     'R^2': R2(y_predicted, np.array(y_test))
                                     }

my_line_regression_classic_errors


{'MAE': 179008316.59116694,
 'RMSE': 201443340.77727112,
 'MSE': 4.0579419543507784e+16,
 'MAPE': 3073744.604686639,
 'R^2': -389329740.1612344}

In [ ]:
def function_optimizer_error(td, cd, w): return gradientDescentAlgorithm(
    td=td,
    cd=cd,
    w=w,
    lost_function=lambda td, cd, w: euclideError(td, cd, w),
    gradient=lambda td, cd, w: euclideErrorGradient(td, cd, w),
    nsteps=30000,
    merror=0.0001,
    step=0.00000000000000001)


regressor = Regression()
trainer = RegressionTeacher(regressor, function_optimizer_error)
trainer.train(Polinomizer(np.array(X_train1, dtype=np.float64), 3).polinomize(),
              np.array(y_train1, dtype=np.float64))

y_predicted = regressor.predict(Polinomizer(
    np.array(X_test1, dtype=np.float64), 3).polinomize())
y_predicted
my_line_regression_classic_errors = {'MAE': MAE(y_predicted, y_test1),
                                     'RMSE': RMSE(y_predicted, y_test1),
                                     'MSE': MSE(y_predicted, y_test1),
                                     'MAPE': MAPE(y_predicted, y_test1),
                                     'R^2': R2(y_predicted, np.array(y_test1))
                                     }

my_line_regression_classic_errors


{'MAE': 34.781174232468395,
 'RMSE': 46.111033309136204,
 'MSE': 2126.227392836269,
 'MAPE': 209.94823802703044,
 'R^2': 0.7828000064710134}